In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA


ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("./clean.csv")

In [ ]:
df = df.drop(columns="Unnamed: 0")

df.columns

In [ ]:
df.head()


In [ ]:

numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
print("Numerical Columns:", numerical_columns)



1.Frequent pattern


In [ ]:
df_drop = df.drop(columns=['Rating', 'Reviews', 'Size', 'Installs',  'Rating Count',
       'Minimum Installs', 'Developer Internal ID'])

In [ ]:
df_drop['Price'] = pd.cut(df_drop['Price'], bins=[-np.inf, 0.33, 0.66, np.inf], labels=['free', 'cheap', 'expensive'], duplicates="drop")

In [ ]:
df_drop=df_drop.drop(columns= [ 'Price', 'Last Updated', 'Current Ver', 'App Id', 'Currency', 
                               'Developer Id', 'Developer Website', 'Developer Email', 'Privacy Policy',
                               'Summary','Developer Address','Android Ver', 'Minimum Android', 'Released','Android version Text',
                               'Developer','Version'])

In [ ]:
df_encoded = pd.get_dummies(df_drop)
df_encoded

In [ ]:
frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)
print(frequent_itemsets)

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
print(rules)


2.clustring


In [ ]:
for col in df_drop.columns:
    print(col , df_drop[col].dtype)

In [ ]:


kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(df_encoded)


In [ ]:
hierarchical = AgglomerativeClustering(n_clusters=4)
hierarchical_labels = hierarchical.fit_predict(df_encoded)


In [5]:
pca = PCA(n_components=2)

visualize = pca.fit_transform(df_encoded)

# Create a scatter plot
plt.figure(figsize=(6, 4))
sns.scatterplot(x=visualize[:, 0], y=visualize[:, 1], hue=kmeans_labels, palette='viridis', s=6)
plt.title('K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

pca = PCA(n_components=2)

visualize = pca.fit_transform(df_encoded)

# Create a scatter plot
plt.figure(figsize=(6, 4))
sns.scatterplot(x=visualize[:, 0], y=visualize[:, 1], hue=hierarchical_labels, palette='viridis', s=6)
plt.title('K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()



NameError: name 'PCA' is not defined

In [ ]:


pca = PCA(n_components=3)
visualize = pca.fit_transform(df_encoded)


fig_kmeans = px.scatter_3d(
    x=visualize[:, 0], 
    y=visualize[:, 1], 
    z=visualize[:, 2], 
    color=kmeans_labels, 
    title='K-Means Clustering',
    size_max=1,
    labels={'x': 'Principal Component 1', 'y': 'Principal Component 2', 'z': 'Principal Component 3'}
)

fig_kmeans.show()

fig_hierarchical = px.scatter_3d(
    x=visualize[:, 0], 
    y=visualize[:, 1], 
    z=visualize[:, 2], 
    color=hierarchical_labels, 
    title='Hierarchical Clustering',
    labels={'x': 'Principal Component 1', 'y': 'Principal Component 2', 'z': 'Principal Component 3'}
)

fig_hierarchical.show()


In [ ]:
df_drop.columns

In [ ]:
df.columns

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer  # وارد کردن enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from scipy import stats


numeric_columns = ['Rating', 'Reviews', 'Size', 'Installs', 'Price', 'Rating Count', 'Minimum Installs']

imputer = IterativeImputer()
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

df['Reviews_per_Install'] = df['Reviews'] / df['Installs']
df['Price_per_Install'] = df['Price'] / df['Installs']
df['log_Reviews'] = np.log1p(df['Reviews'])
df['sqrt_Installs'] = np.sqrt(df['Installs'])

numeric_columns.extend(['Reviews_per_Install', 'Price_per_Install', 'log_Reviews', 'sqrt_Installs'])
df = df[(np.abs(stats.zscore(df[numeric_columns])) < 3).all(axis=1)]

scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

pca = PCA(n_components=2)
pca_components = pca.fit_transform(df[numeric_columns])

selector = SelectKBest(score_func=f_classif, k=5)
selector.fit(df[numeric_columns], df['Rating'])
selected_features = df[numeric_columns].columns[selector.get_support()]

if 'Rating' not in selected_features:
    selected_features = np.append(selected_features, 'Rating')

print("Selected Features:", selected_features)

correlation_matrix = df[selected_features].corr()
plt.figure(figsize=(20, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

target_correlation = correlation_matrix['Rating'].sort_values(ascending=False)
print(target_correlation)


3.classification

In [ ]:


df.fillna(df.mean(), inplace=True)

numeric_columns = ['Rating', 'Reviews', 'Size', 'Installs', 'Price', 'Rating Count', 'Minimum Installs']
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

correlation_matrix = df[numeric_columns].corr()
plt.figure(figsize=(20, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

target_correlation = correlation_matrix['Rating'].sort_values(ascending=False)
print(target_correlation)


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score



In [ ]:
rating_mean = df['Rating'].mean()
rating_std = df['Rating'].std()

def categorize_rating(rating):
    if rating > rating_mean + rating_std:
        return 'High'
    elif rating < rating_mean - rating_std:
        return 'Low'
    else:
        return 'Medium'

df['Rating_Category'] = df['Rating'].apply(categorize_rating)

X = df[selected_features].drop(columns='Rating')
y = df['Rating_Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
rf_classifier = RandomForestClassifier(random_state=42)
svm_classifier = SVC(random_state=42)

classifiers = [
    ('Naive Bayes', nb_classifier),
    ('Random Forest', rf_classifier),
    ('SVM', svm_classifier)
]

evaluation_metrics = {
    'Model': [],
    'Accuracy': [],
    'F1-Score': [],
    'Recall': [],
    'Precision': []
}

for name, classifier in classifiers:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')

    evaluation_metrics['Model'].append(name)
    evaluation_metrics['Accuracy'].append(accuracy)
    evaluation_metrics['F1-Score'].append(f1)
    evaluation_metrics['Recall'].append(recall)
    evaluation_metrics['Precision'].append(precision)

df_metrics = pd.DataFrame(evaluation_metrics)
print(df_metrics)